In [1]:
# Get autocomplete to work
%config Completer.use_jedi = False

# Ensure external Python files are refreshed when reimporting things
%load_ext autoreload
%autoreload 2

In [20]:
from load_functions import load_text
    
text_input = load_text(day=14)
print(len(text_input))
print(text_input[:5])

102
['CVKKFSSNNHNPSPPKBHPB', '', 'OF -> S', 'VO -> F', 'BP -> S']


Convert to initial polymer, and polymer rules

In [21]:
from collections import defaultdict

initial_polymer = text_input[0]

# Turn into dictionary of pairs
list_initial_polymer = list(initial_polymer)

dict_polymer_pairs = defaultdict(int)

for i in range(1, len(list_initial_polymer)):
    pair = list_initial_polymer[i-1] + list_initial_polymer[i]
    dict_polymer_pairs[pair] += 1

# Then construct rules
rules = [rule.split(' -> ') for rule in text_input[2:]]
dict_polymer_rules = {rule[0]: rule[1] for rule in rules}

In [22]:
dict_polymer_pairs['CB']

0

In [23]:
dict_polymer_rules['CB']

'C'

# Problem 

#### Part 1: Run 10 loops of polymer growth, and subtract least frequent letter from most

In [24]:
import copy
import pandas as pd

dict_polymer_pairs_pt1 = copy.deepcopy(dict_polymer_pairs)


def polymer_growth(dict_polymer):
    
    new_dict = copy.deepcopy(dict_polymer)
    
    for pair in dict_polymer.keys():
        n_old_pair = dict_polymer[pair]
        
        # Construct new polymer pairs
        new_element = dict_polymer_rules[pair]
        old_elements = list(pair)
        new_dict[old_elements[0] + new_element] += n_old_pair
        new_dict[new_element + old_elements[1]] += n_old_pair
        
        # And subtract from existing pair
        new_dict[pair] -= n_old_pair
        
    return new_dict

# Loop
for step in range(10):
    dict_polymer_pairs_pt1 = polymer_growth(dict_polymer_pairs_pt1)
    
    
# Convert into elements
def get_elements(dict_polymer):
    
    dict_elements = defaultdict(float)
    
    for pair, num in dict_polymer.items():
        elements = list(pair)
        
        # Prevent double counting by halving numbers added
        dict_elements[elements[0]] += (num / 2)
        dict_elements[elements[1]] += (num / 2)
    
    # Start and end elements of initial polymer won't be double counted, so add 0.5 back
    dict_elements[list_initial_polymer[0]] += 0.5
    dict_elements[list_initial_polymer[-1]] += 0.5
    
    return dict_elements


dict_elements_pt1 = get_elements(dict_polymer_pairs_pt1)

# Convert into dataframe and sort
df_elements = pd.DataFrame.from_dict(dict_elements_pt1, 
                                     orient='index',
                                     columns=['count']).sort_values(by='count')


print('Day 14, part 1:', int(df_elements.sort_values(by='count', ascending=False).iloc[0] - df_elements.iloc[0]))

Day 14, part 1: 2375


#### Part 2: 40 steps

In [26]:
dict_polymer_pairs_pt2 = copy.deepcopy(dict_polymer_pairs)

# Loop
for step in range(40):
    dict_polymer_pairs_pt2 = polymer_growth(dict_polymer_pairs_pt2)
    
dict_elements_pt2 = get_elements(dict_polymer_pairs_pt2)

# Convert into dataframe and sort
df_elements = pd.DataFrame.from_dict(dict_elements_pt2, 
                                     orient='index',
                                     columns=['count']).sort_values(by='count')

print('Day 14, part 2:', int(df_elements.sort_values(by='count', ascending=False).iloc[0] - df_elements.iloc[0]))

Day 14, part 2: 1976896901756
